In [2]:
# Vector Data impimentation
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [3]:
loader = PyPDFLoader("../test_documents/attention.pdf")

In [6]:
document = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
documents =  text_splitter.split_documents(document)
vectors = FAISS.from_documents(documents, embeddings)

In [8]:
retriver = vectors.as_retriever()

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, "Database_Search",
                      """Search for information about the research paper. You should use this tool to get the information about the relevant research 
                      paper.""")

In [11]:
Tools = [retriver_tool]

# Agent Implimentation 

In [14]:
## Agents
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent
from langchain.tools import Tool
from langchain.llms import Ollama
from langchain import hub
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
load_dotenv()


True

In [15]:
llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
llm_with_tools = llm.bind_tools(Tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [36]:
parse_agent_prompt = ChatPromptTemplate.from_template("""You are the Research Document Parsing Agent. 
First Use the given Database_Search Tool to get research paper context and  You Read the given  research paper  Fully and divide them into Abstract, Method, Math, Experiments, Results.
{input}
{agent_scratchpad}
""")

In [37]:
agent = create_tool_calling_agent(llm_with_tools, Tools, parse_agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=Tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [40]:
result_doc_agent=agent_executor.invoke({'input':"analyze the Attention is all you Need given research paper"})



> Entering new AgentExecutor chain...

Invoking: `Database_Search` with `{'query': 'Attention is all you need'}`


around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position ican depend only on the known outputs at positions less than i.
3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
of the values, where the weight assigned to each value is computed by a compatibility function of the
query with the corresponding key.
3.2.1 Scaled Dot-Product Attention
We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of
queries and key

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `openai/gpt-oss-20b` in organization `org_01k26dmhxver3bwfg5g7b19q0b` service tier `on_demand` on tokens per minute (TPM): Limit 8000, Requested 8092, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [41]:
print(output['output'])

**Attention Is All You Need**  
*Vaswani et al., 2017 (arXiv:1706.03762)*  

Below is a full‑paper walk‑through that splits the work into the requested sections, followed by a concise analysis of its key ideas, strengths, and limitations.

---

## 1. Abstract  
> The paper introduces the **Transformer**, a sequence‑to‑sequence model that dispenses with recurrence and convolutions entirely, relying only on a novel **self‑attention** mechanism.  The Transformer achieves state‑of‑the‑art results on WMT 2014 English‑German and English‑French translation tasks while training 3–6× faster than comparable RNN‑based models.  The authors also show that the model scales well to longer sequences and can be trained on a single GPU.

---

## 2. Method (Model Architecture & Training)

| Component | Description | Key Hyper‑parameters |
|-----------|-------------|----------------------|
| **Encoder** | 6 identical layers. Each layer has: <br>• Multi‑head self‑attention (h=8) <br>• Position‑wise feed‑fo